In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib agg

import visualization
from data import datasets, evaluation
import transformations
import jax
import jax.numpy as jnp
import flax.linen as nn
from flax.core import freeze
import optax
import inference
from utils import settings
import numpy as np
import os
import global_settings

rng_key = jax.random.PRNGKey(0)
fraction = 0.2

# Benchmark

In [2]:
dataset_list =[
    os.path.join(
        os.path.join(global_settings.PATH_DATASETS, "benchmark_data"),
        file_name) for file_name in os.listdir("/home/gw/data/datasets/benchmark_data/") if file_name.split('.')[-1] == "data"
]

In [3]:
dataset_indices = {}
for dataset_element in dataset_list:
    print(dataset_element)
    with open(dataset_element, "r") as f:
        lines = f.readlines()
        lines_count = len(lines)
        train_count = int((1.0 - fraction) * lines_count)
        rng_key, rng_key_ = jax.random.split(rng_key)
        indices = jax.random.permutation(rng_key_, lines_count)
        indices_train = np.array(indices[:train_count]).tolist()
        indices_val = np.array(indices[train_count:]).tolist()
        print(len(indices_train), len(indices_val))
        
        # save to dict/json
        dataset_name = dataset_element.split('/')[-1].split('.')[0]
        dataset_indices[dataset_name] = {"train": indices_train, "validate": indices_val}

/home/gw/data/datasets/benchmark_data/naval_compressor.data
9547 2387
/home/gw/data/datasets/benchmark_data/airfoil.data
1202 301
/home/gw/data/datasets/benchmark_data/protein.data
36584 9146
/home/gw/data/datasets/benchmark_data/boston.data
404 102
/home/gw/data/datasets/benchmark_data/diabetes.data
353 89
/home/gw/data/datasets/benchmark_data/video_time.data
55027 13757
/home/gw/data/datasets/benchmark_data/concrete.data
824 206
/home/gw/data/datasets/benchmark_data/yacht.data
246 62
/home/gw/data/datasets/benchmark_data/video_mem.data
55027 13757
/home/gw/data/datasets/benchmark_data/gpu.data
193280 48320
/home/gw/data/datasets/benchmark_data/naval_turbine.data
9547 2387
/home/gw/data/datasets/benchmark_data/forest_fire.data
413 104
/home/gw/data/datasets/benchmark_data/energy.data
614 154
/home/gw/data/datasets/benchmark_data/wine.data
142 36


# Toy

In [14]:
# toy datasets
dataset_indices = {}

## izmailov
dataset_izmailov = datasets.Izmailov()
N = len(dataset_izmailov._data)
print(N, fraction)

rng_key, rng_key_ = jax.random.split(rng_key)
train_count = int((1.0 - fraction) * N)
indices = jax.random.permutation(rng_key_, N)
indices_train = np.array(indices[:train_count]).tolist()
indices_val = np.array(indices[train_count:]).tolist()
print(len(indices_train), len(indices_val))

dataset_name = "izmailov"
dataset_indices[dataset_name] = {"train": indices_train, "validate": indices_val}

## sinusoidal
dataset_izmailov = datasets.Sinusoidal()
N = len(dataset_izmailov._data)
print(N, fraction)

rng_key, rng_key_ = jax.random.split(rng_key)
train_count = int((1.0 - fraction) * N)
indices = jax.random.permutation(rng_key_, N)
indices_train = np.array(indices[:train_count]).tolist()
indices_val = np.array(indices[train_count:]).tolist()
print(len(indices_train), len(indices_val))

dataset_name = "sinusoidal"
dataset_indices[dataset_name] = {"train": indices_train, "validate": indices_val}

400 0.2
320 80
150 0.2
120 30


In [15]:
import json
with open(os.path.join(global_settings.PATH_DATASETS, "toy_dataset_indices_0.2.json"), 'w') as foutput:
    json.dump(dataset_indices, foutput)

In [17]:
test = {
    "a": 1,
    "b": 2
}
print(len(test))

2
